In [1]:
from transformers import AutoTokenizer, AutoModel
from adapters import AutoAdapterModel
import importlib
import plotly.express as px
from sklearn.datasets import load_digits
from umap import UMAP

tokenizer = AutoTokenizer.from_pretrained('allenai/specter2_base')
model = AutoAdapterModel.from_pretrained('allenai/specter2_base')

/Users/arthur/miniconda3/envs/arxiv-daily/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/arthur/miniconda3/envs/arxiv-daily/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/arthur/miniconda3/envs/arxiv-daily/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/arthur/miniconda3/envs/arxiv-daily/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecat

In [2]:
model.load_adapter("allenai/specter2", source="hf", load_as="specter2", set_active=True)

papers = [{'title': 'BERT', 'abstract': 'We introduce a new language representation model called BERT'},
          {'title': 'Attention is all you need', 'abstract': ' The dominant sequence transduction models are based on complex recurrent or convolutional neural networks'}]

# concatenate title and abstract
text_batch = [d['title'] + tokenizer.sep_token + (d.get('abstract') or '') for d in papers]
# preprocess the input
inputs = tokenizer(text_batch, padding=True, truncation=True,
                                   return_tensors="pt", return_token_type_ids=False, max_length=512)
output = model(**inputs)
# take the first token in the batch as the embedding
embeddings = output.last_hidden_state[:, 0, :]

Fetching 4 files: 100%|██████████| 4/4 [00:00<00:00, 33026.02it/s]


In [3]:
embeddings

tensor([[-0.2127,  0.5332, -0.6860,  ..., -0.4872,  0.1453, -0.7420],
        [ 0.2662,  0.5603,  0.0460,  ..., -0.1477, -0.1488,  0.0200]],
       grad_fn=<SliceBackward0>)

In [19]:

import papers
importlib.reload(papers)

manager = papers.PaperManager()
new_papers = manager.daily_paper(3)

print(new_papers[:2])

Fetching 4 files: 100%|██████████| 4/4 [00:00<00:00, 85163.53it/s]


279
[Paper(title='DiffusionMTL: Learning Multi-Task Denoising Diffusion Model from\n  Partially Annotated Data', authors=['Hanrong Ye', 'Dan Xu'], publish_date='2024-03-22T17:59:58Z', abstract='Recently, there has been an increased interest in the practical problem of\nlearning multiple dense scene understanding tasks from partially annotated\ndata, where each training sample is only labeled for a subset of the tasks. The\nmissing of task labels in training leads to low-quality and noisy predictions,\nas can be observed from state-of-the-art methods. To tackle this issue, we\nreformulate the partially-labeled multi-task dense prediction as a pixel-level\ndenoising problem, and propose a novel multi-task denoising diffusion framework\ncoined as DiffusionMTL. It designs a joint diffusion and denoising paradigm to\nmodel a potential noisy distribution in the task prediction or feature maps and\ngenerate rectified outputs for different tasks. To exploit multi-task\nconsistency in denoising

In [20]:
embeddings = manager.paper_embeddings(new_papers[:50])

In [22]:
digits = load_digits()

print(f"{digits.data.shape=}")

umap_2d = UMAP(random_state=0)
umap_2d.fit(embeddings)

projections = umap_2d.transform(embeddings)

fig = px.scatter(
    projections, x=0, y=1
)
fig.show()

digits.data.shape=(1797, 64)


/Users/arthur/miniconda3/envs/arxiv-daily/lib/python3.11/site-packages/umap/umap_.py:1943: UserWarning:

n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.

